# import

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

# data

In [60]:
data_path = (r'..\data\data_ready.csv')
traffic_path = (r'..\data\traffic_data.xlsx')

In [61]:
data = pd.read_csv(data_path,index_col=0)
traffic = pd.read_excel(traffic_path, engine='openpyxl', sheet_name='udaljenosti', index_col=0)

# site i polutanit

In [62]:
sites = ['Nord','Sud','West','Ost','DonBosco'] 
pollutant_O3 = ['N_O3','S_O3'] 
pollutant_PM10 = ['N_PM10K','S_PM10K','W_PM10K','O_PM10K','D_PM10K'] 
pollutant_NO2 = ['N_NO2','S_NO2','W_NO2','O_NO2','D_NO2'] 


In [63]:
PM10 = data.filter(like='PM10K',axis=1)
NO2 = data.filter(like='NO',axis=1)
O3 = data.filter(like='_O3',axis=1)
NOX = data.filter(like='NOX',axis=1)
Ox = data.filter(like='_Ox',axis=1)

In [64]:
test = data.copy()

In [65]:
test = test.drop(PM10,axis=1)
test = test.drop(NO2,axis=1)
test = test.drop(O3,axis=1)
test = test.drop(Ox,axis=1)


In [67]:
sc = StandardScaler()

In [68]:
X_train = test.loc[:'2020-01-02']
X_test = test.loc['2020-01-03':'2020-03-10']

In [69]:
y_train = PM10.loc[:'2020-01-02']
y_test = PM10.loc['2020-01-03':'2020-03-10']

In [70]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [71]:
y_test = y_test.to_numpy()

In [43]:
y_train = y_train.to_numpy()

In [15]:
X_train_t = torch.FloatTensor(X_train)
Y_train_t = torch.FloatTensor(y_train)

In [16]:
X_test_t = torch.FloatTensor(X_test)

In [17]:
# Define network dimensions
input_size = X_train.shape[1]
# Layer size
hidden_size = 1000 # Number of hidden nodes
output_size = 1 # Number of output nodes for prediction

# Build mdel
torch_model = torch.nn.Sequential(torch.nn.Linear(input_size, hidden_size),
                                  torch.nn.Softmax(dim=1),
                                  torch.nn.Linear(hidden_size, output_size))    
print(torch_model)

Sequential(
  (0): Linear(in_features=91, out_features=1000, bias=True)
  (1): Softmax(dim=1)
  (2): Linear(in_features=1000, out_features=1, bias=True)
)


In [18]:
loss_func = torch.nn.MSELoss() #mean square error as loss metric
learning_rate = 0.001
optimizer = torch.optim.Adam(torch_model.parameters(), lr=learning_rate)

In [19]:
train_error = []
epochs = 2000

#Y_train_t = torch.FloatTensor(y_train).reshape(-1,1) #Converting numpy array to torch tensor

for e in range(epochs):
    #X_train_t = torch.FloatTensor(xtrain)  #Converting numpy array to torch tensor
    
    y_pred = torch_model(X_train_t)
    loss = loss_func(y_pred, Y_train_t)
    loss.backward()
    optimizer.step()
    
    optimizer.zero_grad()

    train_error.append(loss.item())



In [20]:
ypredict = torch_model(X_test_t)
# Descale
ypredict_np = ypredict.detach().numpy()
#ypredict_inverse = scaler_y.inverse_transform(ypredict_np)
#ytest_inverse = scaler_y.inverse_transform(ytest)


In [21]:
ypredict.shape

torch.Size([68, 1])

In [22]:
X_test_t.shape

torch.Size([68, 91])

In [23]:
Y_train_t.shape

torch.Size([732, 5])

# random forest

In [72]:
test.columns

Index(['Ost_8010_208', 'DonBosco_8020_110', 'DonBosco_8020_119',
       'DonBosco_8020_122', 'DonBosco_8053_111', 'Nord_8010_205',
       'Ost_8010_209', 'Ost_8010_213', 'Ost_8010_214', 'Nord_8020_102',
       'West_8020_103', 'Sud_8020_107', 'Sud_8041_218', 'Ost_8041_221',
       'Nord_8045_203', 'Nord_8051_101', 'year', 'dayofyear', 'month_Apr',
       'month_Aug', 'month_Dec', 'month_Feb', 'month_Jan', 'month_Jul',
       'month_Jun', 'month_Mar', 'month_May', 'month_Nov', 'month_Oct',
       'month_Sep', 'weekday_Friday', 'weekday_Monday', 'weekday_Saturday',
       'weekday_Sunday', 'weekday_Thursday', 'weekday_Tuesday',
       'weekday_Wednesday', 'season_fall', 'season_spring', 'season_summer',
       'season_winter', 'holiday', 'holiday_school', 'DonBosco_RH',
       'DonBosco_Temp', 'Nord_Precip', 'Nord_Pressure', 'Nord_RH', 'Nord_Temp',
       'Nord_Winddirection', 'Nord_Windspeed', 'Sud_RH', 'Sud_Temp',
       'Sud_Winddirection', 'Sud_Windspeed', 'West_RH', 'West_Temp',
   

In [73]:
model = RandomForestRegressor(random_state=1)

In [74]:
model.fit(X_train,y_train)

RandomForestRegressor(random_state=1)

In [75]:
predictions = model.predict(X_test)

In [76]:
r2 = r2_score(y_test, predictions)
mse = mean_squared_error(y_test, predictions, squared=True)

In [77]:
PM = pd.DataFrame(y_test,columns=PM10.columns)

In [78]:
pred = pd.DataFrame(predictions,columns=PM10.columns)

In [79]:
for i in PM.columns:
    r2 = r2_score(PM[i],pred[i])
    mse = mean_squared_error(PM[i],pred[i], squared=True)
    print(i)
    print(f'R2 {r2}')
    print(f'mse {mse}')
#     plt.plot(PM[i])#_inverse)
#     plt.plot(pred[i])#_inverse)
#     plt.show()
        

D_PM10K
R2 0.5284211313609952
mse 97.44071122519125
N_PM10K
R2 0.6143224645170202
mse 67.83299521837546
O_PM10K
R2 0.39962243133290676
mse 185.3513588978976
S_PM10K
R2 0.6184793560686668
mse 93.50034434364832
W_PM10K
R2 0.6417880156264437
mse 68.29717438759148
